# Nesscary Imports:

In [ ]:
!sudo apt update
!pip install surya-ocr

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Ign:7 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:8 https://r2u.stat.illinois.edu/ubuntu jammy Release
Hit:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
52 packages can be upgraded. Run 'apt list --upgradable' to see them.
W: Skipping acquire of configured file 'main/source/Sources' as 

In [ ]:
import matplotlib.pyplot as plt
import requests
from io import BytesIO
import torch
import numpy as np
import pandas as pd
import os
import multiprocessing
from functools import partial
from tqdm import tqdm
from pathlib import Path
import time
from PIL import Image
import urllib.request
import cv2
import re
from PIL import Image
from surya.ocr import run_ocr
from surya.model.detection.model import load_model as load_det_model, load_processor as load_det_processor
from surya.model.recognition.model import load_model as load_rec_model
from surya.model.recognition.processor import load_processor as load_rec_processor
import requests
from io import BytesIO

# Getting the Dataset and building the nesscary functions

In [ ]:
train_data = pd.read_csv("train_data.csv")
train_data.head()

FileNotFoundError: [Errno 2] No such file or directory: 'train_data.csv'

In [ ]:
def crop_image_into_5_regions(image):
    """Crop an image into 5 regions:
       1. The full image.
       2. Four basic crops of the image (top-left, top-right, bottom-left, bottom-right).
    """
    width, height = image.size
    crops = []

    # Add the full image as the first crop
    crops.append(image)

    # Dimensions for the 4 quadrants
    half_width = width // 2
    half_height = height // 2

    # Define coordinates for the 4 quadrants
    quadrants = [
        (0, 0, half_width, half_height),  # Top-left
        (half_width, 0, width, half_height),  # Top-right
        (0, half_height, half_width, height),  # Bottom-left
        (half_width, half_height, width, height)  # Bottom-right
    ]

    # Crop each quadrant and add to the list
    for coord in quadrants:
        left, upper, right, lower = coord
        cropped_image = image.crop((left, upper, right, lower))
        crops.append(cropped_image)

    return crops


In [ ]:
def process_image_from_url(img_url):
    # Fetch the image from the URL
    response = requests.get(img_url)
    if response.status_code != 200:
        raise ValueError("Error: Unable to fetch image from URL.")

    # Convert the image data to a format suitable for OpenCV
    image_data = BytesIO(response.content)
    image = Image.open(image_data)
    image = cv2.cvtColor(np.array(image), cv2.COLOR_RGB2BGR)

    # Check if the image is loaded correctly
    if image is None:
        raise ValueError("Error: Image not found or unable to load.")

    # Convert the image to grayscale
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Apply Gaussian Blur to reduce noise
    blurred_image = cv2.GaussianBlur(gray_image, (5, 5), 0)

    # Apply adaptive thresholding to enhance text
    threshold_image = cv2.adaptiveThreshold(
        blurred_image, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
        cv2.THRESH_BINARY, 11, 2
    )

    # Create a sharpening kernel
    sharp_kernel = np.array([[-1, -1, -1],
                             [-1,  9, -1],
                             [-1, -1, -1]])

    # Apply the sharpening filter
    sharpened_image = cv2.filter2D(threshold_image, -1, sharp_kernel)

    # Perform dilation to make text more pronounced
    kernel = np.ones((1, 1), np.uint8)
    dilated_image = cv2.dilate(sharpened_image, kernel, iterations=1)

    # Perform erosion to remove small noise
    eroded_image = cv2.erode(dilated_image, kernel, iterations=1)

    # Convert the processed image to PIL format
    processed_image_pil = Image.fromarray(cv2.cvtColor(eroded_image, cv2.COLOR_BGR2RGB))

    return processed_image_pil

In [ ]:
stop_words = {"and", "at", "for", "the", "a", "is", "in", "on"}

def preprocess_text(text):
    # 2. Replace multiple spaces (including newlines) with a single space, then strip leading and trailing spaces
    text = re.sub(r'\s+', ' ', text).strip()

    # 3. Remove any character that is not a word character (a-z, A-Z, 0-9, _) or whitespace or specific punctuation (.-)
    text = re.sub(r'[^\w\s.-]', '', text)

    # 4. Replace multiple spaces again with a single space if needed
    text = re.sub(r'\s+', ' ', text)

    # 5. Split the text into words, remove stop words, and rejoin the cleaned text
    words = text.split()  # Split text into individual words
    filtered_words = [word for word in words if word not in stop_words]  # Filter out stop words

    # 6. Return the cleaned and filtered text
    return ' '.join(filtered_words)

In [ ]:
def ocr_text(url):

  generated_text = " "  # Initialize as an empty string
  processed_image = process_image_from_url(url)

  # Crop the image into 9 grids
  crops = crop_image_into_5_regions(processed_image)

  for image in crops:

    langs = ["en"]  # Add more languages if needed

    # Load models and processors
    det_processor, det_model = load_det_processor(), load_det_model()
    rec_model, rec_processor = load_rec_model(), load_rec_processor()

    # Run OCR on the image
    predictions = run_ocr([image], [langs], det_model, det_processor, rec_model, rec_processor)

    # Extract the entire text from predictions
    full_text = " "

    # Iterate through predictions
    for prediction in predictions:
        # Check if the result has 'text_lines'
        if hasattr(prediction, 'text_lines'):
            text_lines = prediction.text_lines
            for text_line in text_lines:
                full_text += text_line.text + " "
    return preprocess_text(full_text)


In [ ]:
text = ocr_text("https://m.media-amazon.com/images/I/61I9XdN6OFL.jpg")
print(text)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/675 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/811 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/154M [00:00<?, ?B/s]

Loaded detection model vikp/surya_det3 on device cuda with dtype torch.float16


...

generation_config.json:   0%|          | 0.00/163 [00:00<?, ?B/s]

Loaded recognition model vikp/surya_rec2 on device cuda with dtype torch.float16


preprocessor_config.json:   0%|          | 0.00/556 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Recognizing Text: 100%|██████████| 1/1 [00:01<00:00,  1.79s/it]

INGRÉDIENT MÉNAGER MULTI-USAGE TERRE DE SOMMIÈRES 100 NATUREL Argile 100 pure et naturelle la terre de Sommières présente des propriétés absorbances qui permettent le nettoyage à sec des lâches récaklurances sur coures les surfaces moquette tapis parquet . Elle est aussi efficace pour désodortser le linge. Dosage conseille Selo


In [ ]:
text

'INGRÉDIENT MÉNAGER MULTI-USAGE TERRE DE SOMMIÈRES 100 NATUREL Argile 100 pure et naturelle la terre de Sommières présente des propriétés absorbances qui permettent le nettoyage à sec des lâches récaklurances sur coures les surfaces moquette tapis parquet . Elle est aussi efficace pour désodortser le linge. Dosage conseille Selo'

In [ ]:
img_link_list = train_data["image_link"]

NameError: name 'train_data' is not defined

In [ ]:
textual_description = []

In [ ]:
textual_description = []

for i in range(2):

  img_link = img_link_list[i]
  ocr_text_result = ocr_text(img_link)
  textual_description.append(ocr_text_result)


print(f"Textual Description: {textual_description}")


[2024/09/15 13:39:49] ppocr DEBUG: Namespace(help='==SUPPRESS==', use_gpu=False, use_xpu=False, use_npu=False, use_mlu=False, ir_optim=True, use_tensorrt=False, min_subgraph_size=15, precision='fp32', gpu_mem=500, gpu_id=0, image_dir=None, page_num=0, det_algorithm='DB', det_model_dir='/root/.paddleocr/whl/det/en/en_PP-OCRv3_det_infer', det_limit_side_len=960, det_limit_type='max', det_box_type='quad', det_db_thresh=0.3, det_db_box_thresh=0.6, det_db_unclip_ratio=1.5, max_batch_size=10, use_dilation=False, det_db_score_mode='fast', det_east_score_thresh=0.8, det_east_cover_thresh=0.1, det_east_nms_thresh=0.2, det_sast_score_thresh=0.5, det_sast_nms_thresh=0.2, det_pse_thresh=0, det_pse_box_thresh=0.85, det_pse_min_area=16, det_pse_scale=1, scales=[8, 16, 32], alpha=1.0, beta=1.0, fourier_degree=5, rec_algorithm='SVTR_LCNet', rec_model_dir='/root/.paddleocr/whl/rec/en/en_PP-OCRv4_rec_infer', rec_image_inverse=True, rec_image_shape='3, 48, 320', rec_batch_num=6, max_text_length=25, rec_c

In [ ]:
textual_description[1]

'EARRIFIC LEBENSMITTELECHT GEPRAGTES DESIGN AS YOULIKE SEST. Designedin Berlin LIZENZIERTEAUNDIGESCHUTZTEIDESIGNS TEARRIFIC LEBENSMITTELECHT ASYOULIKE OR LIZENZIERTEAUNDG GEPRAGTES DESIGN BESI Designedin Berlin SCHUTZTEDESIGNS'

In [ ]:
len(textual_description)

2

# Extracting the OCR of the test dataset:

In [ ]:
test_data = pd.read_csv('test_data.csv')
test_data.head()

,index,image_link,group_id,entity_name
0,0,https://m.media-amazon.com/images/I/110EibNycl...,156839,height
1,1,https://m.media-amazon.com/images/I/11TU2clswz...,792578,width
2,2,https://m.media-amazon.com/images/I/11TU2clswz...,792578,height
3,3,https://m.media-amazon.com/images/I/11TU2clswz...,792578,depth
4,4,https://m.media-amazon.com/images/I/11gHj8dhhr...,792578,depth


In [ ]:
test_image_links = test_data["image_link"]

In [ ]:
test_image_links.shape

(131187,)

In [ ]:
textual_description = []

In [ ]:
len(textual_description)

0

In [ ]:
for i in range(100):

  img_link = test_image_links[i+3000]
  ocr_text_result = ocr_text(img_link)
  textual_description.append(ocr_text_result)

Loaded detection model vikp/surya_det3 on device cuda with dtype torch.float16


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Loaded recognition model vikp/surya_rec2 on device cuda with dtype torch.float16


Recognizing Text: 100%|██████████| 1/1 [00:00<00:00,  1.96it/s]


Loaded detection model vikp/surya_det3 on device cuda with dtype torch.float16
Loaded recognition model vikp/surya_rec2 on device cuda with dtype torch.float16


...

Recognizing Text: 100%|██████████| 1/1 [00:01<00:00,  1.75s/it]


Loaded detection model vikp/surya_det3 on device cuda with dtype torch.float16
Loaded recognition model vikp/surya_rec2 on device cuda with dtype torch.float16


Recognizing Text: 100%|██████████| 1/1 [00:00<00:00,  2.98it/s]


Loaded detection model vikp/surya_det3 on device cuda with dtype torch.float16
Loaded recognition model vikp/surya_rec2 on device cuda with dtype torch.float16


Recognizing Text: 100%|██████████| 1/1 [00:01<00:00,  1.51s/it]


In [ ]:
len(textual_description)

100

In [ ]:
test_data_with_text = test_data.iloc[:200]

In [ ]:
len(textual_description)

200

In [ ]:
test_data_with_text.shape

(200, 4)

In [ ]:
test_data_with_text.loc[:, 'Textual Description'] = textual_description


In [ ]:
test_data_with_text.head()

,index,image_link,group_id,entity_name,Textual Description
0,0,https://m.media-amazon.com/images/I/110EibNycl...,156839,height,
1,1,https://m.media-amazon.com/images/I/11TU2clswz...,792578,width,Ize Width Length One Size 42cm16.64 200cm78.74...
2,2,https://m.media-amazon.com/images/I/11TU2clswz...,792578,height,Ize Width Length One Size 42cm16.64 200cm78.74...
3,3,https://m.media-amazon.com/images/I/11TU2clswz...,792578,depth,Ize Width Length One Size 42cm16.64 200cm78.74...
4,4,https://m.media-amazon.com/images/I/11gHj8dhhr...,792578,depth,Size Wdth Length One Slze 10.50cm4.13 90cm35.4...


In [ ]:
test_data_with_text.to_csv('test_data_with_text.csv', index=False)

In [ ]:
from google.colab import files

# Download the file
files.download('test_data_with_text.csv')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
import requests

def get_bad_request_indices(image_links_list):
  """
  Finds the indices of image links that return a 400 Bad Request.

  Args:
    img_links: A list of image URLs.

  Returns:
    A list of indices corresponding to bad requests.
  """

  bad_request_indices = []
  for i, img_link in enumerate(image_links_list):
    try:
      response = requests.get(img_link)
      response.raise_for_status()  # Raise an exception for error status codes
    except requests.exceptions.HTTPError as e:
      if e.response.status_code == 400:
        bad_request_indices.append(i)

  return bad_request_indices

In [ ]:
get_bad_request_indices(test_image_links)

KeyboardInterrupt: 